Поиск ассоциативных правил

In [1]:
pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=28fd50620d5a9229c9991843566ac0bfb9f11fbfd876c1dfb5494a0a9f5d5967
  Stored in directory: /root/.cache/pip/wheels/77/3d/a6/d317a6fb32be58a602b1e8c6b5d6f31f79322da554cad2a5ea
Successfully built apyori


In [2]:
import apyori
from apyori import apriori
import numpy as np
import pandas as pd
import plotly.express as px

In [4]:
ds=pd.read_csv('Sleep_health_and_lifestyle_prep.csv')
ds=ds.drop(['Unnamed: 0'],axis=1)
ds.head()

,Gender,Age,Occupation,Sleep_Duration,Quality_of_Sleep,Physical_Activity_Level,Stress_Level,BMI_Category,Heart_Rate,Daily_Steps,Sleep_Disorder,Systolic_BP,Diastoli_BP
0,1,27,9,6.1,6,42,6,2,77,4200,1,126.0,83.0
1,1,28,1,6.2,6,60,8,0,75,10000,1,125.0,80.0
2,1,28,1,6.2,6,60,8,0,75,10000,1,125.0,80.0
3,1,28,6,5.9,4,30,8,1,85,3000,2,140.0,90.0
4,1,28,6,5.9,4,30,8,1,85,3000,2,140.0,90.0


In [6]:
#наблюдение 1 - большая часть пациентов с нормальным весом, за которыми идут пациенты  с избыточным
x = ds['BMI_Category'].value_counts().sort_values(ascending=False)[:10]
fig = px.bar(x= x.index, y= x.values)
fig.update_layout(title_text= "Top 10 frequently BMI Category (Tabular Representation)", xaxis_title= "Patient_bmi", yaxis_title="Count")
fig.show()

In [8]:
# наблюдение 2 - Manager, Salesperson, Scientist имеют избыточный вес (отнесены только к классу Overweight),
# на основании средней численности шагов
pvt=ds.pivot_table(values='Daily_Steps', columns=['Occupation'], index='BMI_Category', aggfunc=np.mean)
pvt

<ipython-input-8-3a575c1df919>:3: FutureWarning:

The provided callable <function mean at 0x7af1fad62b60> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.



Occupation,0,1,2,3,4,5,6,7,8,9,10
BMI_Category,,,,,,,,,,,
0,7051.612903,7000.0,6016.666667,7941.860465,NaN,5285.714286,NaN,NaN,NaN,8000.0,7000.000000
1,NaN,3600.0,NaN,3300.000000,NaN,NaN,3000.0,NaN,NaN,3000.0,3500.000000
2,6000.000000,NaN,5266.666667,6000.000000,5500.0,8351.515152,NaN,6000.0,5350.0,4200.0,5842.424242


In [9]:
# вместо NaN проставим 0
pvt[np.isnan(pvt)] = 0
# все значения>0 заменим 1, чтобы больше проявить правила (не обязательно, но повышает точность)
pvt[pvt>0]=1
pvt


Occupation,0,1,2,3,4,5,6,7,8,9,10
BMI_Category,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


In [10]:
def transaction_list(df):
    list_external=[]
    for i in range(df.shape[0]):
        list_internal=[]
        data=df.iloc[i]
        index=data[data>0]
        for element in index.index:
            list_internal.append(element)
        list_external.append(list_internal)
    return list_external

transactions=transaction_list(pvt[:100])
transactions

[[0, 1, 2, 3, 5, 9, 10], [1, 3, 6, 9, 10], [0, 2, 3, 4, 5, 7, 8, 9, 10]]

In [11]:
rules = apriori(transactions, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)
association_results = list(rules)
print(association_results)

[RelationRecord(items=frozenset({4, 7}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset({4}), items_add=frozenset({7}), confidence=1.0, lift=3.0), OrderedStatistic(items_base=frozenset({7}), items_add=frozenset({4}), confidence=1.0, lift=3.0)]), RelationRecord(items=frozenset({8, 4}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset({4}), items_add=frozenset({8}), confidence=1.0, lift=3.0), OrderedStatistic(items_base=frozenset({8}), items_add=frozenset({4}), confidence=1.0, lift=3.0)]), RelationRecord(items=frozenset({8, 7}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset({7}), items_add=frozenset({8}), confidence=1.0, lift=3.0), OrderedStatistic(items_base=frozenset({8}), items_add=frozenset({7}), confidence=1.0, lift=3.0)]), RelationRecord(items=frozenset({0, 4, 7}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset({4}), items_add=fr

In [13]:
for item in association_results:
    pair = item[0]
    items = [x for x in pair]

    # Преобразуйте элементы в строки перед объединением
    print('Rule: ' + str(items[0]) + ' -> ' + str(items[1]))
    print('Support: ' + str(item[1]))
    print('Confidence: ' + str(item[2][0][2]))
    print('Lift: ' + str(item[2][0][3]))
    print('______________________________')


Rule: 4 -> 7
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
______________________________
Rule: 8 -> 4
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
______________________________
Rule: 8 -> 7
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
______________________________
Rule: 0 -> 4
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
______________________________
Rule: 0 -> 8
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
______________________________
Rule: 0 -> 8
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
______________________________
Rule: 2 -> 4
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
______________________________
Rule: 8 -> 2
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
______________________________
Rule: 8 -> 2
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
______________________________
Rule: 3 -> 4
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
______________________________
Rule: 8 -> 3
Support

In [14]:
pip install --upgrade mlxtend

In [15]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [16]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.666667,(0)
1,0.666667,(1)
2,0.666667,(2)
3,1.000000,(3)
4,0.333333,(4)
...,...,...
586,0.333333,"(0, 2, 3, 5, 7, 8, 9, 10)"
587,0.333333,"(0, 2, 4, 5, 7, 8, 9, 10)"
588,0.333333,"(0, 3, 4, 5, 7, 8, 9, 10)"
589,0.333333,"(2, 3, 4, 5, 7, 8, 9, 10)"


In [17]:
# сгенерируем ассоциативные правила с уровнем доверия 0.2
#данный уровень поддержки довольно низкий и используется только для примера
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/association_rules.py:186: RuntimeWarning:

invalid value encountered in divide



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(0),(1),0.666667,0.666667,0.333333,0.500000,0.75,1.0,-0.111111,0.666667,-0.5,0.333333,-0.50,0.500000
1,(1),(0),0.666667,0.666667,0.333333,0.500000,0.75,1.0,-0.111111,0.666667,-0.5,0.333333,-0.50,0.500000
2,(0),(2),0.666667,0.666667,0.666667,1.000000,1.50,1.0,0.222222,inf,1.0,1.000000,1.00,1.000000
3,(2),(0),0.666667,0.666667,0.666667,1.000000,1.50,1.0,0.222222,inf,1.0,1.000000,1.00,1.000000
4,(0),(3),0.666667,1.000000,0.666667,1.000000,1.00,1.0,0.000000,inf,0.0,0.666667,0.00,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,(5),"(0, 2, 3, 4, 7, 8, 9, 10)",0.666667,0.333333,0.333333,0.500000,1.50,1.0,0.111111,1.333333,1.0,0.500000,0.25,0.750000
20116,(7),"(0, 2, 3, 4, 5, 8, 9, 10)",0.333333,0.333333,0.333333,1.000000,3.00,1.0,0.222222,inf,1.0,1.000000,1.00,1.000000
20117,(8),"(0, 2, 3, 4, 5, 7, 9, 10)",0.333333,0.333333,0.333333,1.000000,3.00,1.0,0.222222,inf,1.0,1.000000,1.00,1.000000
20118,(9),"(0, 2, 3, 4, 5, 7, 8, 10)",1.000000,0.333333,0.333333,0.333333,1.00,1.0,0.000000,1.000000,0.0,0.333333,0.00,0.666667


In [18]:
# найдем ассоциативные правила с уровнем независимости больше 1.2
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(0),(2),0.666667,0.666667,0.666667,1.0,1.5,1.0,0.222222,inf,1.0,1.0,1.00,1.00
1,(2),(0),0.666667,0.666667,0.666667,1.0,1.5,1.0,0.222222,inf,1.0,1.0,1.00,1.00
2,(0),(4),0.666667,0.333333,0.333333,0.5,1.5,1.0,0.111111,1.333333,1.0,0.5,0.25,0.75
3,(4),(0),0.333333,0.666667,0.333333,1.0,1.5,1.0,0.111111,inf,0.5,0.5,1.00,0.75
4,(0),(5),0.666667,0.666667,0.666667,1.0,1.5,1.0,0.222222,inf,1.0,1.0,1.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16951,(2),"(0, 3, 4, 5, 7, 8, 9, 10)",0.666667,0.333333,0.333333,0.5,1.5,1.0,0.111111,1.333333,1.0,0.5,0.25,0.75
16952,(4),"(0, 2, 3, 5, 7, 8, 9, 10)",0.333333,0.333333,0.333333,1.0,3.0,1.0,0.222222,inf,1.0,1.0,1.00,1.00
16953,(5),"(0, 2, 3, 4, 7, 8, 9, 10)",0.666667,0.333333,0.333333,0.5,1.5,1.0,0.111111,1.333333,1.0,0.5,0.25,0.75
16954,(7),"(0, 2, 3, 4, 5, 8, 9, 10)",0.333333,0.333333,0.333333,1.0,3.0,1.0,0.222222,inf,1.0,1.0,1.00,1.00


In [21]:
w = pd.ExcelWriter('AssocRules_SHL.xlsx')
rules.to_excel(w, sheet_name='AR')
w.close()
